In [1]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\larry\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                               4.10.0-py38haa244fe_1 --> 4.10.1-py38haa244fe_0




conda-4.10.1         | 3.1 MB    |            |   0% 
conda-4.10.1         | 3.1 MB    |            |   1% 
conda-4.10.1         | 3.1 MB    | #2         |  13% 
conda-4.10.1         | 3.1 MB    | ##8        |  29% 
conda-4.10.1         | 3.1 MB    | ####5      |  45% 
conda-4.10.1         | 3.1 MB    | ######1    |  61% 
conda-4.10.1         | 3.1 MB   

In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
#pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.display.precision = 20

import heapq
from collections import defaultdict

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# RESET AIR

In [4]:
a, i, r = (pd.read_csv("student_movie_attributes.csv"),
           pd.read_csv("student_movie_intent.csv"),
           pd.read_csv("student_movie_ratings.csv"))

In [5]:
a, i, r = (a.drop(['etl_run_guid', 'intent_score', 'survey_date', 'content_type', 'movie_name',
                   'household_size', 'gender_name', 'age_group_bracket', 'ethnicity_name',
                   'education_level', 'income_bracket'], axis=1),
           i.drop(['etl_run_guid'], axis=1),
           r.drop(['etl_run_guid', 'networks', 'content_type'], axis=1))

In [6]:
a, i, r = (a.sort_values(by=['respondent_id', 'movie_id']),
           i.sort_values(by=['respondent_id', 'movie_id']),
           r.sort_values(by=['respondent_id', 'movie_id']))

In [7]:
ir = r.merge(i, how='outer', on=['respondent_id', 'movie_id'])

In [8]:
air = ir.merge(a, how='outer', on=['respondent_id', 'movie_id'])
air = air.dropna()

print('(Rows, Columns)', air.shape)
print('Any null values?', air.isnull().values.any())
air

(Rows, Columns) (249284, 74)
Any null values? False


,survey_date,respondent_id,movie_id,movie_name,gender_name,age_group_bracket,ethnicity_name,education_level,income_bracket,household_size,rating,is_favorite,respondent_movie_id,all_soon_as_available,all_looking_forward,all_total_attention,all_best_option,all_no_rewatch,all_yes_rewatch,all_none_above,all_aware_before_release,home_background,home_surfing_channels,home_go_to,home_sig_other,home_no_phone,home_yes_phone,home_bits_pieces,social_talk_others,social_yes_recommend,social_none_above,intent_raw,intent_score,wave_id,age,zip_code,live_with_children,watch_last,intelligent,real,fun,touching,relatable,sad,romantic,informative,mean,fake,glamorous,original,predictable,fresh,mindless,unrelatable,funny,imaginative,wholesome,inspiring,shallow,edgy,fastpaced,disturbing,dramatic,outrageous,exciting,calming,slow,unoriginal,uplifting,sexy,deep,suspenseful,thrilling,scary
0,1/18/2020,743006,11549,Ad Astra,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,4,0,1464319,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.70307010000000003114,0.11021033908869472262,191.0,64.0,85378.0,False,3 to 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1/18/2020,743006,11679,Beetlejuice,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,5,0,1464318,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.70307010000000003114,0.11021033908869472262,191.0,64.0,85378.0,False,3 to 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1/18/2020,743006,11692,Deliverance,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,5,0,1464315,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-0.85973709999999992082,0.07689052718983381551,191.0,64.0,85378.0,False,In the last year,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,1/18/2020,743006,11708,Gremlins,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,5,0,1464324,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-0.85973709999999992082,0.07689052718983381551,191.0,64.0,85378.0,False,3 to 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1/18/2020,743006,11748,Risky Business,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,3,0,1464323,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,-0.85973709999999992082,0.07689052718983381551,191.0,64.0,85378.0,False,In the last year,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256299,6/11/2020,956985,9111,Abducted: The Carlina White Story,Female,65 and Over,African-American or Black,High school diploma,"$25,000 - $34,999",1,4,0,1787030,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.21399999999999996803,0.66925988225399501985,193.0,69.0,28429.0,True,1 to 3 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
256301,6/12/2020,957051,9111,Abducted: The Carlina White Story,Female,65 and Over,White or Caucasian,High school diploma,"$15,000 - $24,999",2,2,0,1787032,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.59499999999999997335,0.28889823380992435409,193.0,69.0,19111.0,False,3 to 6 months,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
256302,6/11/2020,957162,9111,Abducted: The Carlina White Sto

In [9]:
air.movie_name.nunique()

3213

# STOP RESET

# COLLABORATIVE FILTERING

In [10]:
cf = air[['respondent_id', 'movie_name', 'rating']]
cf

,respondent_id,movie_name,rating
0,743006,Ad Astra,4
1,743006,Beetlejuice,5
2,743006,Deliverance,5
3,743006,Gremlins,5
4,743006,Risky Business,3
...,...,...,...
256299,956985,Abducted: The Carlina White Story,4
256301,957051,Abducted: The Carlina White Story,2
256302,957162,Abducted: The Carlina White Story,4
256303,957166,Abducted: The Carlina White Story,3


In [11]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cf[['respondent_id', 'movie_name', 'rating']], reader)
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': False}  # compute cosine similarities between items
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [12]:
cf.movie_name.nunique()

3213

In [13]:
movies = cf.movie_name.unique()
resp = cf.respondent_id.unique()

In [14]:
for movie in movies: 
    print(movie, algo.predict(783008, movie).est)

Ad Astra 3.958709744708846
Beetlejuice 3.958709744708846
Deliverance 3.958709744708846
Gremlins 3.958709744708846
Risky Business 3.958709744708846
The Devil's Advocate 3.958709744708846
The Dirty Dozen 3.958709744708846
Demolition Man 3.958709744708846
Get Smart 3.958709744708846
GoodFellas 3.958709744708846
Flowers in the Attic 3.958709744708846
Last Christmas 3.958709744708846
Deep Blue Sea 3.958709744708846
Friday 3.958709744708846
Mad Max Beyond Thunderdome 3.958709744708846
Passenger 57 3.958709744708846
Drew Peterson: Untouchable 3.958709744708846
Aaliyah: The Princess of R&B 3.958709744708846
Death of a Cheerleader 3.958709744708846
Menendez: Blood Brothers 3.958709744708846
The Craigslist Killer 3.958709744708846
Child's Play 3.958709744708846
Yesterday 3.958709744708846
One Flew Over the Cuckoo's Nest 3.958709744708846
Executive Decision 3.958709744708846
A Quiet Place 3.958709744708846
The Godfather 3.958709744708846
The Amazing Spider-Man 3.958709744708846
Annabelle Comes Ho

Let Me In 3.958709744708846
The Grudge 3.958709744708846
The Conjuring 3.958709744708846
Fifty Shades of Grey 3.958709744708846
Grosse Pointe Blank 3.958709744708846
Saw III 3.958709744708846
Never Say Never Again 3.958709744708846
Fantastic Four: Rise of the Silver Surfer 3.958709744708846
Good Will Hunting 3.958709744708846
Mechanic: Resurrection 3.958709744708846
Rain Man 3.958709744708846
Face/Off 3.958709744708846
There's Something About Mary 3.958709744708846
Pocahontas 3.958709744708846
Big Momma's House 2 3.958709744708846
New Nightmare 3.958709744708846
It 3.958709744708846
Resting Place 3.958709744708846
The Stepford Wives 3.958709744708846
Abraham Lincoln: Vampire Hunter 3.958709744708846
Fast & Furious 3.958709744708846
Predator 3.958709744708846
Robin Hood 3.958709744708846
The Incredible Hulk 3.958709744708846
Cocktail 3.958709744708846
Bad Boys 3.958709744708846
Bringing Down the House 3.958709744708846
Last Holiday 3.958709744708846
The Best Man Holiday 3.95870974470884

Popstar: Never Stop Never Stopping 3.958709744708846
The Game Plan 3.958709744708846
Wanderlust 3.958709744708846
The Last Sharknado: It's About Time 3.958709744708846
Pride and Prejudice and Zombies 3.958709744708846
The Choice 3.958709744708846
The Last Airbender 3.958709744708846
Gravity 3.958709744708846
Honey, I Shrunk the Kids 3.958709744708846
Jerry Maguire 3.958709744708846
X-Men: First Class 3.958709744708846
Basic Instinct 3.958709744708846
Dark Passage 3.958709744708846
A History of Violence 3.958709744708846
The Hunt for Red October 3.958709744708846
Serendipity 3.958709744708846
Batman Begins 3.958709744708846
Joe Dirt 2: Beautiful Loser 3.958709744708846
The Heartbreak Kid 3.958709744708846
The Princess Diaries 3.958709744708846
Johnny English 3.958709744708846
Tarzan 3.958709744708846
Breakheart Pass 3.958709744708846
Open Range 3.958709744708846
Polar 3.958709744708846
The Deer Hunter 3.958709744708846
The Green Hornet 3.958709744708846
Solo: A Star Wars Story 3.9587097

Braddock: Missing in Action III 3.958709744708846
Rounders 3.958709744708846
Over Her Dead Body 3.958709744708846
The Living Daylights 3.958709744708846
16 Blocks 3.958709744708846
Ghost Ship 3.958709744708846
Jackie 3.958709744708846
Paper Towns 3.958709744708846
Meet John Doe 3.958709744708846
Napoleon Dynamite 3.958709744708846
The Longest Day 3.958709744708846
Pinocchio 3.958709744708846
Requiem for a Dream 3.958709744708846
Exit Strategy 3.958709744708846
The Huntsman: Winter's War 3.958709744708846
The Pink Panther 3.958709744708846
Dude, Where's My Car? 3.958709744708846
You Only Live Twice 3.958709744708846
Knock Knock 3.958709744708846
The Miseducation of Cameron Post 3.958709744708846
Case 39 3.958709744708846
The English Patient 3.958709744708846
Mad Max 3.958709744708846
My Giant 3.958709744708846
Captains Courageous 3.958709744708846
Grandma 3.958709744708846
Abducted: The Carlina White Story 3.958709744708846
The Gift 3.958709744708846
21 & Over 3.958709744708846
Mother's

Kansas City Confidential 3.958709744708846
Dreamcatcher 3.958709744708846
Meet Boston Blackie 3.958709744708846
The Shepherd of the Hills 3.958709744708846
Riot in Cell Block 11 3.958709744708846
The Private Life of Henry VIII 3.958709744708846
The Southerner 3.958709744708846
The Red Badge of Courage 3.958709744708846
The Roman Spring of Mrs. Stone 3.958709744708846
The Command 3.958709744708846
Rushmore 3.958709744708846
The Wild and the Innocent 3.958709744708846
Bees Make Honey 3.958709744708846
In Harm's Way 3.958709744708846
The Rains of Ranchipur 3.958709744708846
White Nights 3.958709744708846
The Overbrook Brothers 3.958709744708846
Submission 3.958709744708846
The Fighting Vigilantes 3.958709744708846
The Shoes of the Fisherman 3.958709744708846
Volunteers 3.958709744708846
Lumberjack 3.958709744708846
Think Like a Dog 3.958709744708846
Artemis Fowl 3.958709744708846
The King of Staten Island 3.958709744708846
My Spy [2020] 3.958709744708846
Hamilton [2020] 3.958709744708846


# RECOMMENDATION FUNCTION

In [15]:
#TOP 10 RECOMMENDATIONS

def recommendation_10(ID):
    pd.options.display.precision = 2
    rcmd = pd.DataFrame(columns=['Movie_Name', 'Rating'])

    for movie in movies:
        rcmd = rcmd.append({'Movie_Name': movie, 'Rating': algo.predict(ID, movie).est}, ignore_index=True)
     
        rcmd = rcmd.sort_values(by=['Rating'], ascending=False).head(10)
    title = 'Respondent ' + str(ID) + "'s Top 10 Recommended Movies", rcmd
    return title

In [16]:
# Input ID Range from 743006 to 791017

recommendation_10(743006)

("Respondent 743006's Top 10 Recommended Movies",
                            Movie_Name  Rating
 0        Sharknado 5: Global Swarming     5.0
 1                   National Treasure     5.0
 2  National Treasure: Book of Secrets     5.0
 3                  Return of the Jedi     5.0
 4              A Good Day to Die Hard     5.0
 5                     The Lone Ranger     5.0
 6              Boo! A Madea Halloween     5.0
 7                               Sully     5.0
 8                          The Intern     5.0
 9                        Wonder Woman     5.0)

In [17]:
#RECOMMENDS MOVIES WITH RATINGS = 5

def recommendation_5(ID):
    pd.options.display.max_rows = None
    pd.options.display.precision = 2
    rcmd = pd.DataFrame(columns=['Movie_Name', 'Rating'])

    for movie in movies:
        rcmd = rcmd.append({'Movie_Name': movie, 'Rating': algo.predict(ID, movie).est}, ignore_index=True)
        rcmd = rcmd.sort_values(by=['Rating'], ascending=False)
        rcmd = rcmd.loc[rcmd['Rating'] == 5]
    title = 'Respondent ' + str(ID) + ':' + ' ' + str(len(rcmd.index)) +' Movies Likely to Rate 5',rcmd
    return title

In [18]:
# Input ID Range from 743006 to 791017

recommendation_5(743006)

('Respondent 743006: 738 Movies Likely to Rate 5',
                                                          Movie_Name  Rating
 0                                      Sharknado 5: Global Swarming     5.0
 496                                                  Les Misérables     5.0
 487                                        Kubo and the Two Strings     5.0
 488                                                        Hercules     5.0
 489                                                            Sing     5.0
 490                                                 The Maze Runner     5.0
 491                                               The Bourne Legacy     5.0
 492                                                     Ratatouille     5.0
 493                                                 The Perfect Guy     5.0
 494                                                       Project X     5.0
 495                                         The Wolf of Wall Street     5.0
 497                     

In [19]:
pd.options.display.max_rows = 10